# TCC WASM -  Análise Estatiscas

Análise dos dados de execução do processamento de detecção de faces compilados com WebAssembly versus compilação nativa com visual C++. O objetivo desta pesqusias em constatar a diferença de performance entre um código compilado p/ WebAssembly e outro com compilaçção nativa.

## Ferramentas para analise

In [2]:
import pandas as pd
import os

## Dataset

Conjuntos de dado usados nos testes são videos codiifcados em MP4 opvtidos no portal de videos publicos YouTube.
Todos os videos 

## Resultados

Foram feitos testes com uma aplicação nativa usado o Windows10 e com WebAssembly usando Chrome 75 e Firefox 67 executando no mesmo sistema operacional e maquina fisica com as configuraçãoes padrão de instalação. 

In [4]:
df_dataset = pd.read_csv('./data/dataset.csv')
df_dataset

,id,extension,resolution,frames,fps,youtube_link
0,DATASET_1,mp4,1920x1080,3915,24,https://www.youtube.com/watch?v=EWUdGRAwUpY
1,DATASET_2,mp4,1920x1080,3020,30,https://www.youtube.com/watch?v=vQtLX6pW5eA
2,DATASET_3,mp4,1920x1080,12296,50,https://www.youtube.com/watch?v=RuL5jVqc4Tg


In [5]:
df_meta_fd = pd.read_csv('./data/META_LOG_FACE_DETECT.csv')
df_meta_fd

,workload,dataset,resolution,system,total_frames,total_frames_processeds,test_total_time_ms,log_file
0,FACE_DETECT,DATASET_1,SMALL,Chrome_75_Windows,3915,775,169977.790,LOG_FACE_DETECT_DATASET_1_SMALL_Chrome_75_Wind...
1,FACE_DETECT,DATASET_1,SMALL,Firefox_67_Windows,3915,885,163392.000,LOG_FACE_DETECT_DATASET_1_SMALL_Firefox_67_Win...
2,FACE_DETECT,DATASET_1,SMALL,WINDOWS_10_WIN32,3915,1845,167644.000,LOG_FACE_DETECT_DATASET_1_SMALL_WINDOWS_10_WIN...
3,FACE_DETECT,DATASET_1,MEDIUM,Chrome_75_Windows,3915,756,350346.115,LOG_FACE_DETECT_DATASET_1_MEDIUM_Chrome_75_Win...
4,FACE_DETECT,DATASET_1,MEDIUM,Firefox_67_Windows,3915,445,163044.000,LOG_FACE_DETECT_DATASET_1_MEDIUM_Firefox_67_Wi...
5,FACE_DETECT,DATASET_1,MEDIUM,WINDOWS_10_WIN32,3915,1738,242765.000,LOG_FACE_DETECT_DATASET_1_MEDIUM_WINDOWS_10_WI...
6,FACE_DETECT,DATASET_1,LARGE,Chrome_75_Windows,3915,750,783297.990,LOG_FACE_DETECT_DATASET_1_LARGE_Chrome_75_Wind...
7,FACE_DETECT,DATASET_1,LARGE,Firefox_67_Windows,3915,198,163618.000,LOG_FACE_DETECT_DATASET_1_LARGE_Firefox_67_Win...
8,FACE_DETECT,DATASET_1,LARGE,WINDOWS_10_WIN32,3915,1935,547485.000,LOG_FACE_DETECT_DATASET_1_LARGE_WINDOWS_10_WIN...
9,FACE_DETECT,DATASET_2,SMALL,Chrome_75_Windows,3020,735,190739.770,LOG_FACE_DETECT_DATASET_2_SMALL_Chrome_75_Wind...


## Dataset 1 - Small